## MAKING THE STUDENTS ASSISMENT MODEL

In [1]:
print("hello world")

hello world


In [2]:
import numpy as np

In [18]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#MODELING
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
# import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('stud.csv')

In [5]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
X= df.drop('math_score',axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [7]:
y = df['math_score']
y.head()    

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [19]:
print("catagories in gender column: ",end=" ")
print(df['gender'].unique())


print("catagories in race column: ",end=" ")
print(df['race_ethnicity'].unique())


print("catagories in education of parents column: ",end=" ")
print(df['parental_level_of_education'].unique())


print("catagories in luch column: ",end=" ")
print(df['lunch'].unique())


print("catagories in test perception column: ",end=" ")
print(df['test_preparation_course'].unique())

catagories in gender column:  ['female' 'male']
catagories in race column:  ['group B' 'group C' 'group A' 'group D' 'group E']
catagories in education of parents column:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
catagories in luch column:  ['standard' 'free/reduced']
catagories in test perception column:  ['none' 'completed']


In [22]:
# Recreate X from df to ensure it's a dataframe (in case of out-of-order execution)
X = df.drop('math_score', axis=1)

numeric_features = X.select_dtypes(exclude=['object']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numric_transformers = StandardScaler()
oh_transformers = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("onehotEncoder", oh_transformers, categorical_features),
        ("standardScaler", numric_transformers, numeric_features)
    ]
)

In [23]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [24]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)

X_train shape:  (800, 19)
X_test shape:  (200, 19)


In [29]:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    rmse = np.sqrt(mse)
    return mae, mse, r2, rmse

In [30]:
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),  
    "Support Vector Regressor": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "CatBoost Regressor": CatBoostRegressor(random_state=42, verbose=0),
    "XGBoost Regressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(random_state=42),
    "Lasso": Lasso(random_state=42),
    "Ridge": Ridge(random_state=42)
}

In [32]:
model_list = []
r2_list =[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train model

    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_mse, model_train_r2, model_train_rmse = evaluate_model(y_train, y_train_pred)

    model_test_mae, model_test_mse, model_test_r2, model_test_rmse = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

Linear Regression
Model performance for Training set
- Root Mean Squared Error: 5.3231
- Mean Absolute Error: 4.2667
- R2 Score: 0.8743
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 5.3940
- Mean Absolute Error: 4.2148
- R2 Score: 0.8804


Random Forest Regressor
Model performance for Training set
- Root Mean Squared Error: 2.3048
- Mean Absolute Error: 1.8325
- R2 Score: 0.9764
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 6.0177
- Mean Absolute Error: 4.6649
- R2 Score: 0.8512


Decision Tree Regressor
Model performance for Training set
- Root Mean Squared Error: 0.2795
- Mean Absolute Error: 0.0187
- R2 Score: 0.9997
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 7.7146
- Mean Absolute Error: 6.1950
- R2 Score: 0.7554


Support Vector Regressor
Model performance for Training set
- Root Mean Squared Error: 6.5770
- Mean Absolute Error: 4.8692
- R